In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("train_interaction.txt",header=None,sep='\t')
data.columns = ['u_id','photo_id','click','like','follow','time','play_time','duration_time']
data.describe()

In [ ]:
data = data[['u_id','photo_id','click','time']]
data.shape

In [ ]:
data['time'].max()

In [ ]:
import time

# 输入毫秒级的时间，转出正常格式的时间
def timeStamp(timeNum):
    timeStamp = float(timeNum/100)
    timeArray = time.localtime(timeStamp)
    otherStyleTime = time.strftime("%Y-%m-%d %H:%M:%S", timeArray)
    print(otherStyleTime)
timeStamp(data['time'].max())

In [ ]:
timeStamp(data['time'].min())

# only keep positive samples

In [ ]:
data = data[data['click'].isin([1])]
data.shape

In [ ]:
def filter_g_k_one(data,k=10,u_name='user_id',i_name='business_id',y_name='stars'):
    item_group = data.groupby(i_name).agg({y_name:'count'})
    item_g10 = item_group[item_group[y_name]>=10].index
    data_new = data[data[i_name].isin(item_g10)]
    user_group = data_new.groupby(u_name).agg({y_name:'count'})
    user_g10 = user_group[user_group[y_name]>=10].index
    data_new = data_new[data_new[u_name].isin(user_g10)]
    return data_new

def filter_tot(data,k=10,u_name='user_id',i_name='business_id',y_name='stars'):
    data_new=data
    while True:
        data_new = filter_g_k_one(data_new,k=k,u_name=u_name,i_name=i_name,y_name=y_name)
        m1 = data_new.groupby(i_name).agg({y_name:'count'})
        m2 = data_new.groupby(u_name).agg({y_name:'count'})
        num1 = m1[y_name].min()
        num2 = m2[y_name].min()
        print('item min:',num1,'user min:',num2)
        if num1>=k and num2>=k:
            break
    return data_new

In [ ]:
data_10core = filter_tot(data,k=10,u_name='u_id',i_name='photo_id',y_name='click')
data_10core.shape

In [ ]:
user_num = data_10core['u_id'].unique().shape[0]
user_num
item_num = data_10core['photo_id'].unique().shape[0]
item_num
itr_num = data_10core.shape[0]*1.
sparse =  itr_num / (user_num*item_num)
sparse
print("user:",user_num,'item_num:',item_num,'sparse:',sparse)
print("avg itr of user:",itr_num/user_num,'avg itr of item:',itr_num/item_num)

In [ ]:
data = data_10core
data = data.sort_values(by=['time'],ignore_index=True)
slot_num = 10
time_span = (data['time'].max() - data['time'].min())//slot_num
time_min = data['time'].min()
data['time_slot'] = data['time'].apply(lambda x: min((x-time_min)//time_span,slot_num-1))

In [ ]:
data.tail(5)

In [ ]:
train_slot  = round(slot_num * 0.9)
data_train = data[data['time_slot'].isin(list(range(0,train_slot)))]
data_test = data[data['time_slot'].isin(list(range(train_slot,slot_num)))]
print("train number:",data_train.shape,"data_test:",data_test.shape)

filter new user/item in test set, and drop repeated itr in train and item

In [ ]:
user_in_train = data_train['u_id'].unique()
item_in_train = data_train['photo_id'].unique()

data_test = data_test[data_test['u_id'].isin(user_in_train)]
print("user not include in user_items_test:",data_test.shape)
data_test = data_test[data_test['photo_id'].isin(item_in_train)]
print("train:",data_train.shape[0],'not-new test:',data_test.shape[0])

data_train = data_train.drop_duplicates(subset=['u_id','photo_id'],keep='first')
data_test = data_test.drop_duplicates(subset=['u_id','photo_id'],keep='first')
print("not repeat train:",data_train.shape[0],'not-repeat test:',data_test.shape[0])

In [ ]:
print("user in train:",user_in_train.shape,'item in train:',item_in_train.shape)

In [ ]:
import numpy as np
user = data_train['u_id'].unique()
item= data_train['photo_id'].unique()
user_to_id = dict(zip(list(user),list(np.arange(user.shape[0]))))
item_to_id = dict(zip(list(item),list(range(item.shape[0]))))
print("user num:",user.shape)
print("item num:", item.shape)
data_train['uid'] = data_train['u_id'].map(user_to_id)
data_train['iid'] = data_train['photo_id'].map(item_to_id)

data_test['uid'] = data_test['u_id'].map(user_to_id)
data_test['iid'] = data_test['photo_id'].map(item_to_id)

data_test.head(2)

# tot stage data

In [ ]:
data_train = data_train[['uid','iid','time_slot','click']]
data_test = data_test[['uid','iid','time_slot','click']]

In [ ]:
data_train.shape

In [ ]:
data_test.shape

In [ ]:
mm = pd.concat([data_train,data_test],axis=0,ignore_index=True)
mm.shape

In [ ]:
mm.head(4)

In [ ]:
mm.tail(5)
# mm['time_slot'] = mm['time_slot'].astype(int)
# mm['click'] = mm['click'].astype(int)
mm = mm [['uid','iid','click','time_slot']]
mm.columns = ['user_id:token', 'item_id:token', 'rating:float','timestamp:float']
mm.head(5)

In [ ]:
mm.to_csv("kwai.inter",index=False,sep=' ')

In [ ]:
m=data_test.groupby('uid').agg({'iid':'count'})
m.head(2)

In [ ]:
m.describe()

In [ ]:
data_train.shape

# popularity any

# saving ...

In [ ]:
data_train.to_csv('train_interaction_filter.txt',header=False,index=False,sep=' ')
data_test.to_csv('teset_interaction_filter.txt',header=None,index=False,sep=' ')

In [ ]:
data_train.shape

In [ ]:
user_clicked_items = data_train.sort_values(by='uid',ignore_index=True)
print(user_clicked_items.shape)
user_clicked_items.head(2)

In [ ]:
train_itr = user_clicked_items.values[:,0:2]
train_itr.shape

In [ ]:
with open('train.txt','w') as f:
    u_pre = train_itr[0,0]
    k = 0
    for x in train_itr:
        u = x[0]
        i = x[1]
        if u !=u_pre or k==0:
            u_pre = u
            if k>0:
                f.write('\n')
            f.write(str(u))
            k = 1
        f.write(' '+str(i))

In [ ]:
user_items_test = data_test.sort_values(by='uid',ignore_index=True)
print(user_items_test.head(2))
test_itr = user_items_test.values[:,0:2]
test_itr.shape
with open('test.txt','w') as f:
    u_pre = test_itr[0,0]
    k = 0
    for x in test_itr:
        u = x[0]
        i = x[1]
        if u !=u_pre or k==0:
            u_pre = u
            if k>0:
                f.write('\n')
            f.write(str(u))
            k = 1
        f.write(' '+str(i))

In [ ]:
data_finetune = data_train[data_train['time_slot'].isin([train_slot-1])]
user_items_test = data_finetune.sort_values(by='uid',ignore_index=True)
print(user_items_test.head(2))
print('time slot:',data_finetune['time_slot'].unique())
test_itr = user_items_test.values[:,0:2]
test_itr.shape
with open('./fine_tune/train.txt','w') as f:
    u_pre = test_itr[0,0]
    k = 0
    for x in test_itr:
        u = x[0]
        i = x[1]
        if u !=u_pre or k==0:
            u_pre = u
            if k>0:
                f.write('\n')
            f.write(str(u))
            k = 1
        f.write(' '+str(i))

In [ ]:
for slot_id in range(slot_num-1):
    slot_data = data_train[data_train['time_slot'].isin([slot_id])]
    slot_data  = slot_data.sort_values(by=['iid'],ignore_index=True)
    slot_data_np = slot_data[['iid','uid']].values[:,0:2]
    print(slot_data.head(2))
    print(slot_data[['iid','uid']].head(2))
    print(slot_data.shape)
    with open("t_"+str(slot_id)+".txt",'w') as f:
        i_pre = slot_data_np[0,0]
        k = 0
        for x in slot_data_np:
            i_ = x[0]
            u_ = x[1]
            if i_ != i_pre or k == 0:
                i_pre = i_
                if k>0:
                    f.write('\n')
                f.write(str(i_))
                k = 1
            f.write(" " + str(u_))

In [ ]:
slot_test = data_test.sort_values('iid',ignore_index=True)
slot_data_np = slot_test[['iid','uid']].values[:,0:2]
print(slot_test[['iid','uid']].head(2))
print(slot_test.shape)
slot_id = slot_num-1
with open("t_"+str(slot_id)+".txt",'w') as f:
    i_pre = slot_data_np[0,0]
    k = 0
    for x in slot_data_np:
        i_ = x[0]
        u_ = x[1]
        if i_ != i_pre or k == 0:
            i_pre = i_
            if k>0:
                f.write('\n')
            f.write(str(i_))
            k = 1
        f.write(" " + str(u_))



